In [1]:
import numpy as np
import pandas as pd 
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense

/home/rathanak/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/rathanak/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rathanak/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rathanak/anaconda3/lib/python3.6/si

In [2]:
data_kh = pd.read_csv("data/data_kh.csv", header=None)
data_rom = pd.read_csv("data/data_rom.csv", header=None)
batch_size = 32  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 7154  # Number of samples to train on.
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()


In [3]:
for input_text in data_kh[0]:
    input_text = str(input_text).strip()
    input_texts.append(input_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)

for target_text in data_rom[0]:
    target_text = '\t' + str(target_text).strip() + '\n'
    target_texts.append(target_text)
    for char in str(target_text):
        if char not in target_characters:
            target_characters.add(char)

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 17911
Number of unique input tokens: 76
Number of unique output tokens: 70
Max sequence length for inputs: 30
Max sequence length for outputs: 47


In [4]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

print(input_token_index)
print(target_token_index)

print(encoder_input_data)
print(decoder_input_data)
print(decoder_target_data)

{'ែ': 0, 'ប': 1, 'ល': 2, 'ថ': 3, 'ជ': 4, 'ឲ': 5, 'ា': 6, 'ិ': 7, 'ព': 8, 'រ': 9, 'េ': 10, 'ត': 11, 'ួ': 12, 'ញ': 13, 'ើ': 14, 'ទ': 15, 'ធ': 16, '័': 17, 'ឪ': 18, 'ំ': 19, 'ឍ': 20, '៎': 21, 'ឰ': 22, '៏': 23, 'ឥ': 24, 'ច': 25, 'ផ': 26, 'ង': 27, 'ៅ': 28, '់': 29, 'ឋ': 30, 'ឯ': 31, 'ឿ': 32, 'ី': 33, 'ឮ': 34, 'ោ': 35, 'ះ': 36, 'ណ': 37, 'វ': 38, 'ឆ': 39, 'ុ': 40, '៊': 41, '៉': 42, '៌': 43, 'ឫ': 44, 'ៃ': 45, 'គ': 46, 'ឦ': 47, 'អ': 48, 'ន': 49, 'ខ': 50, 'ឬ': 51, 'ឳ': 52, 'ឹ': 53, 'ហ': 54, 'យ': 55, 'ៀ': 56, '្': 57, 'ក': 58, 'ឡ': 59, 'ស': 60, 'ឱ': 61, 'ឌ': 62, 'ឭ': 63, 'ឈ': 64, 'ៈ': 65, '\u200b': 66, 'ភ': 67, 'ដ': 68, '៍': 69, 'ឧ': 70, '?': 71, 'ូ': 72, 'ឺ': 73, 'ម': 74, 'ឃ': 75}
{'်': 0, 'ရ': 1, 'F': 2, 'ဲ': 3, 'း': 4, 'ု': 5, 'g': 6, 'ာ': 7, 's': 8, 'o': 9, 't': 10, 'I': 11, 'd': 12, '?': 13, '\n': 14, 'E': 15, 'ပ': 16, '\t': 17, 'T': 18, 'ွ': 19, '့': 20, 'C': 21, 'p': 22, 'ဖ': 23, 'q': 24, 'm': 25, 'ေ': 26, 'e': 27, 'ျ': 28, 'S': 29, 'န': 30, 'b': 31, 'x': 32, 'L': 33, 'ီ': 34, 'ဂ': 35, '/'

In [5]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    #encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
    
# print(encoder_input_data[0])
# print(decoder_input_data[0])
# print(decoder_target_data[0])
print(num_encoder_tokens)

76


In [7]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 14328 samples, validate on 3583 samples
Epoch 1/100
14328/14328 [==============================] - 81s 6ms/step - loss: 0.5840 - acc: 0.8418 - val_loss: 0.5407 - val_acc: 0.8366
Epoch 2/100
14328/14328 [==============================] - 76s 5ms/step - loss: 0.4432 - acc: 0.8664 - val_loss: 0.5123 - val_acc: 0.8445
Epoch 3/100
14328/14328 [==============================] - 77s 5ms/step - loss: 0.4264 - acc: 0.8701 - val_loss: 0.4950 - val_acc: 0.8482
Epoch 4/100
14328/14328 [==============================] - 74s 5ms/step - loss: 0.4127 - acc: 0.8739 - val_loss: 0.4844 - val_acc: 0.8510
Epoch 5/100
14328/14328 [==============================] - 75s 5ms/step - loss: 0.4013 - acc: 0.8772 - val_loss: 0.4749 - val_acc: 0.8537
Epoch 6/100
14328/14328 [==============================] - 75s 5ms/step - loss: 0.3907 - acc: 0.8805 - val_loss: 0.4677 - val_acc: 0.8558
Epoch 7/100
14328/14328 [==============================] - 75s 5ms/step - loss: 0.3806 - acc: 0.8833 - val_loss: 0.4605 - v

14328/14328 [==============================] - 78s 5ms/step - loss: 0.2205 - acc: 0.9294 - val_loss: 0.6039 - val_acc: 0.8603
Epoch 60/100
14328/14328 [==============================] - 74s 5ms/step - loss: 0.2193 - acc: 0.9299 - val_loss: 0.6074 - val_acc: 0.8600
Epoch 61/100
14328/14328 [==============================] - 69s 5ms/step - loss: 0.2183 - acc: 0.9300 - val_loss: 0.6082 - val_acc: 0.8593
Epoch 62/100
14328/14328 [==============================] - 69s 5ms/step - loss: 0.2171 - acc: 0.9304 - val_loss: 0.6152 - val_acc: 0.8593
Epoch 63/100
14328/14328 [==============================] - 69s 5ms/step - loss: 0.2160 - acc: 0.9308 - val_loss: 0.6144 - val_acc: 0.8597
Epoch 64/100
14328/14328 [==============================] - 69s 5ms/step - loss: 0.2149 - acc: 0.9311 - val_loss: 0.6249 - val_acc: 0.8598
Epoch 65/100
14328/14328 [==============================] - 71s 5ms/step - loss: 0.2137 - acc: 0.9314 - val_loss: 0.6217 - val_acc: 0.8592
Epoch 66/100
14328/14328 [==============

In [8]:
# Save model
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save('s2s.h5')

/home/rathanak/anaconda3/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [9]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [10]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [19]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
#     if str(decoded_sentence).strip() != str(target_texts[seq_index]).strip():
#         print('-')
#         print('KH: ' + input_texts[seq_index] + ', Roman: ' + target_texts[seq_index] + ', predicted: ' + decoded_sentence)
    print('KH: ' + input_texts[seq_index] + ', Roman: ' + target_texts[seq_index] + ', predicted: ' + decoded_sentence)
#     print('Decoded sentence:', decoded_sentence)
#     print('Actual sentence:', target_texts[seq_index])
#     print(str(decoded_sentence).strip() == str(target_texts[seq_index]).strip())

KH: បាន, Roman: 	ban
, predicted: cheatephoumeasa

KH: ការ, Roman: 	kar
, predicted: bangaeb ngkot

KH: មាន, Roman: 	mean
, predicted: angkhuosa

KH: ជា, Roman: 	chea
, predicted: cheaksteng

KH: នៅ, Roman: 	now
, predicted: rotneakar

KH: និង, Roman: 	ning
, predicted: pheang

KH: ដែល, Roman: 	del
, predicted: dauch

KH: ក្នុង, Roman: 	knong
, predicted: kng

KH: នេះ, Roman: 	nih
, predicted: sami

KH: ថា, Roman: 	tha
, predicted: kaun

KH: ទៅ, Roman: 	tow
, predicted: kanhchungk

KH: ពី, Roman: 	pi
, predicted: phnek

KH: មិន, Roman: 	min
, predicted: mean

KH: របស់, Roman: 	robsa
, predicted: brayeang

KH: ទី, Roman: 	ti
, predicted: teang

KH: មក, Roman: 	mk
, predicted: saamrechchett

KH: ឲ្យ, Roman: 	aoy
, predicted: cheab

KH: នោះ, Roman: 	noh
, predicted: kraoyke bangaosa

KH: ហើយ, Roman: 	haey
, predicted: khosa

KH: តែ, Roman: 	te
, predicted: srabok

KH: សម, Roman: 	sam
, predicted: samont

KH: លោក, Roman: 	lok
, predicted: teangoasa

KH: ដោយ, Roman: 	daoy
, predicted: touch